In [1]:
# Import dependencies
import os
import glob
import time
import datetime
import re
from re import search
import copy # might not need this
import pandas as pd
from splinter import Browser
import requests
from bs4 import BeautifulSoup as soup
import random
#import pymongo

In [2]:
def return_dt():
    global current_date
    current_date = str(datetime.datetime.now()).replace(':','.').replace(' ','_')[:-7]
    return current_date
#return_dt()

In [3]:
 executable_path = {'executable_path': './chromedriver.exe'}

In [4]:
 url = input("Step 1) Please copy and paste your laptop query that you want to webscrape, and press enter: ")

Step 1) Please copy and paste your laptop query that you want to webscrape, and press enter: https://www.newegg.com/Gaming-Desktops/SubCategory/ID-3742?Tid=897483


In [5]:
# Step 2)

# Function to ask users if they want to watch the Bot (headless = False) work OR not (headless = True)
# Lastly, will take you directly to the webpage that was inputted
head = ''
browser =''
def head_on_off(executable_path):
    # Have moved two preset variables, head and browser that are both " = '' "
    # assigning these as global variables enable us to reference them outside and inside the function
    global head
    global browser
    # options creates a bound to an answer
    options = [1, 2]
    #executable_path = {'executable_path': './chromedriver.exe'}
    # for all cases where users input in a value that is not valid
    while head not in options:
        head = int(input('Do you want the desktop watch the bot work? Enter a number: 1 - YES | 2 - NO . Your Answer: '))
        if head not in options:
            print("That was not a valid answer. Please try again. ")
    # For cases where users enter in valid options:
    if head == options[0]:
        print('Head is activated. Please view only the new automated Google Chrome web browser. ')
        print('Do not make any adjustments to this automated window while the program runs, as it may produce errors or undesired outputs. ')
        browser = Browser('chrome', **executable_path, headless=False)
    if head == options[1]:
        print('Headless mode activated. No web browser will pop up. Please proceeed. ')
        browser = Browser('chrome', **executable_path, headless=True)
    # visit the target site
    browser.visit(url)
    global current_url
    current_url = browser.url
    #print(current_url)
    return current_url

head_on_off(executable_path)
#time.sleep(5)

Do you want the desktop watch the bot work? Enter a number: 1 - YES | 2 - NO . Your Answer: 1
Head is activated. Please view only the new automated Google Chrome web browser. 
Do not make any adjustments to this automated window while the program runs, as it may produce errors or undesired outputs. 


'https://www.newegg.com/Gaming-Desktops/SubCategory/ID-3742?Tid=897483'

In [6]:
# Step 3)

# Use Splinter to grab the current url, to setup request to pull URL
#current_url = browser.url #+ '&Page=' #+ str(turn_page)

# Use Request.get() to pull the current url
current_url = browser.url
print(current_url)

https://www.newegg.com/Gaming-Desktops/SubCategory/ID-3742?Tid=897483


In [7]:
response = requests.get(current_url)
response

<Response [200]>

In [8]:
# Step 4)

# Use BeautifulSoup to grab all the HTML using the htmlparser
current_page_soup = soup(response.text, 'html.parser')

In [9]:
current_page_soup

<!DOCTYPE HTML>

<html lang="en-us">
<head>
<title>Gaming Desktop PCs - Newegg.com</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="always" name="referrer"/>
<meta content="Gaming Desktops" name="keywords"/>
<meta content="Play all your favorite high-performance gaming titles with speed and high-quality graphical displays on specifically designed gaming desktop PCs." name="description"/>
<meta content="https://c1.neweggimages.com/WebResource/Themes/2005/Nest/logo_424x210.png" property="og:image"/>
<meta content="Play all your favorite high-performance gaming titles with speed and high-quality graphical displays on specifically designed gaming desktop PCs." property="og:description"/>
<link href="https://www.newegg.com/amp/Gaming-Desktops/SubCategory/ID-3742" rel="amphtml"/>
<meta content="english" name="language"/>
<meta content="© 2000-2020 Newegg Inc." name="copyright"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="i

In [ ]:
#current_page_soup.find_all('a', class_="item-title")[0]['href']

In [10]:
current_page_soup.find_all('div', class_="nav-x-body-top-bar fix")[0].text.split('\n')[5]

'Computer Systems'

In [11]:
current_page_soup.find_all('h1', class_="page-title-text")[0].text

'Gaming Desktops'

In [ ]:
#current_page_soup.find_all("a", class_="item-title")[0].text
#current_page_soup.find_all("div", class_="item-container")

#example = current_page_soup.find_all('a', class_="item-title")[0]['href'].split('p/')[1].split('?')[0]

try:
    example1 = current_page_soup.find_all('a', class_="item-title")[0]['href'].split('p/')[1].split('?')[0]
    print("example 1")
    print(example1)
except (IndexError) as e:
    example2 = current_page_soup.find_all('a', class_="item-title")[0]['href'].split('p/')[1]
    print("example 2")
    print(example2)
#bool(example.split('?'))

# if bool(example.split('?')) == True:
#     #example.split('?')
    
# if re.search('?', example) == True:
#     print(True)

In [ ]:
example.split('?')[0]

In [ ]:
current_page_soup.find_all('a', class_="item-title")[0]['href'].split('p/')[1]

In [ ]:
# Step 5) Are there scrappable items-contrainers on the page? List first, last and count, also how many pages

def scrappable_y_n(current_page_soup):
    global containers
    containers = current_page_soup.find_all("div", class_="item-container")
    
    # print first and last objects so users can understand what the output will be
    print("Preview: expect these scrapped off this page, and for every other total results pages, if there's more than one: ")
    print("="*35)
    # max items should be 36
    counter = 0
    for con in containers:
        try:
            counter += 1
            product_details = con.find_all("a", class_="item-title")[0].text
            product_price = con.find_all("li", class_="price-current")[0].text.split()[0]
            print(f'{counter}) {product_details} | Price: {product_price}')
            print("-"*35)
            
        except (IndexError) as e:
            print(f"{counter}) This item was not scrappable. Skipped. ")
            print("-"*35)
            
    print("="*60)
    if counter == 0:
        print("Unable to scrap this link. ")
    else:
        print(f"{len(containers)} Scrappable Objects on the page. ")
    #return containers
#scrappable_y_n(current_page_soup)

In [ ]:
# Create basic classes here and then have the function create product objects AND export out to CSV

In [12]:
class Product_catalog:
    
    all_prod_count = 0
    
    def __init__(self, general_category): # computer systems
        self.general_category = general_category
        
        Product_catalog.all_prod_count += 1
        
    def count_prod(self):
        return int(self.all_prod_count)
        #return '{}'.format(self.general_category)
        
class Sub_category(Product_catalog): # laptops/notebooks, gaming
    
    sub_category_ct = 0
    
    def __init__(self, general_category, sub_categ, item_num, brand, price, img_link, prod_link, model_specifications, current_promotions):
        super().__init__(general_category)
        Sub_category.sub_category_ct += 1
        
        self.sub_categ = sub_categ
        self.item_num = item_num
        self.brand = brand
        self.price = price
        self.img_link = img_link
        self.prod_link = prod_link
        self.model_specifications = model_specifications
        self.current_promotions = current_promotions

        

In [13]:
# TEST CREATING OBJECTS AND IT WORKS

lptp_1 = Sub_category(
"Computer_Systems",
"Laptops/Notebooks",
'Item=9SIA7AB8D73120',
"HP",
1449.00, "//c1.neweggimages.com/ProductImageCompressAll300/A7AB_1_201811092013621813.jpg",
'https://www.newegg.com/p/1TS-000D-032G0?Item=9SIA7AB8D73120',
                      
'HP EliteBook 840 G5 Premium School and Business Laptop (Intel 8th Gen i7-8550U Quad-Core, 16GB RAM, 256GB PCIe SSD, 14" FHD 1920x1080 Sure View Display, Thunderbolt3, NFC, Fingerprint, Win 10 Pro)',   
'Free Expedited Shipping')


#Product_catalog.all_prod_count

In [14]:
# learning purposes
Sub_category.__dict__

mappingproxy({'__module__': '__main__',
              'sub_category_ct': 1,
              '__init__': <function __main__.Sub_category.__init__(self, general_category, sub_categ, item_num, brand, price, img_link, prod_link, model_specifications, current_promotions)>,
              '__doc__': None})

In [ ]:
#Product_catalog.__dict__
print(Product_catalog.count_prod)

In [ ]:
# need to pass in the Sub_category for it to know what to count
Product_catalog.count_prod(Sub_category)

In [ ]:
lptp_1.count_prod()

In [ ]:
Sub_category.sub_category_ct

In [43]:
keys_list = list(lptp_1.__dict__.keys())
keys_list

['general_category',
 'sub_categ',
 'item_num',
 'brand',
 'price',
 'img_link',
 'prod_link',
 'model_specifications',
 'current_promotions']

In [23]:
lptp_1.__dict__.values()

dict_values(['Computer_Systems', 'Laptops/Notebooks', 'Item=9SIA7AB8D73120', 'HP', 1449.0, '//c1.neweggimages.com/ProductImageCompressAll300/A7AB_1_201811092013621813.jpg', 'https://www.newegg.com/p/1TS-000D-032G0?Item=9SIA7AB8D73120', 'HP EliteBook 840 G5 Premium School and Business Laptop (Intel 8th Gen i7-8550U Quad-Core, 16GB RAM, 256GB PCIe SSD, 14" FHD 1920x1080 Sure View Display, Thunderbolt3, NFC, Fingerprint, Win 10 Pro)', 'Free Expedited Shipping'])

In [44]:
dict_values_list = list(lptp_1.__dict__.values())
dict_values_list

['Computer_Systems',
 'Laptops/Notebooks',
 'Item=9SIA7AB8D73120',
 'HP',
 1449.0,
 '//c1.neweggimages.com/ProductImageCompressAll300/A7AB_1_201811092013621813.jpg',
 'https://www.newegg.com/p/1TS-000D-032G0?Item=9SIA7AB8D73120',
 'HP EliteBook 840 G5 Premium School and Business Laptop (Intel 8th Gen i7-8550U Quad-Core, 16GB RAM, 256GB PCIe SSD, 14" FHD 1920x1080 Sure View Display, Thunderbolt3, NFC, Fingerprint, Win 10 Pro)',
 'Free Expedited Shipping']

In [48]:
df1 = pd.DataFrame({
    'keys': keys_list,
    'values': dict_values_list
})

In [51]:
df1.transpose()

,0,1,2,3,4,5,6,7,8
keys,general_category,sub_categ,item_num,brand,price,img_link,prod_link,model_specifications,current_promotions
values,Computer_Systems,Laptops/Notebooks,Item=9SIA7AB8D73120,HP,1449,//c1.neweggimages.com/ProductImageCompressAll3...,https://www.newegg.com/p/1TS-000D-032G0?Item=9...,HP EliteBook 840 G5 Premium School and Busines...,Free Expedited Shipping


In [ ]:


# handy trick - to see all data from the object.
#print(prod_1.__dict__)
#prod_1

lptp_1 = sub_category("Computer_Systems", Product_catalog.all_prod_count,"Laptops/Notebooks", sub_category.sub_cat_ct, "HP", 1449.00, "//c1.neweggimages.com/ProductImageCompressAll300/A7AB_1_201811092013621813.jpg", 'https://www.newegg.com/p/1TS-000D-032G0?Item=9SIA7AB8D73120',
'HP EliteBook 840 G5 Premium School and Business Laptop (Intel 8th Gen i7-8550U Quad-Core, 16GB RAM, 256GB PCIe SSD, 14" FHD 1920x1080 Sure View Display, Thunderbolt3, NFC, Fingerprint, Win 10 Pro)', 'Free Expedited Shipping')
lptp_1.__dict__

In [ ]:
prod_2.__dict__

In [ ]:
# reenabled item_number
def newegg_page_scraper(containers, turn_page): #before: (containers, turn_page)
    images = []
    product_brands = []
    product_models = []
    product_links = []
    item_numbers = []
    product_categories = []
    promotions = []
    prices = []
    shipping_terms = []
    page_nums = []


    for con in containers:
        try:
            page_counter = turn_page
            page_nums.append(int(turn_page))
            
            image = con.a.img["src"]
            #print(image)
            images.append(image)

            prd_title = con.find_all('a', class_="item-title")[0].text
            product_models.append(prd_title)

            product_link = con.find_all('a', class_="item-title")[0]['href']
            product_links.append(product_link)
            
        try:
            item_num = current_page_soup.find_all('a', class_="item-title")[0]['href'].split('p/')[1].split('?')[0]
            item_numbers.append(item_num)
        except (IndexError) as e:
            item_num = current_page_soup.find_all('a', class_="item-title")[0]['href'].split('p/')[1]
            item_numbers.append(item_num)
            
            shipping = con.find_all('li', class_='price-ship')[0].text.strip().split()[0]
            if shipping != "Free":
                shipping = shipping.replace('$', '')
                shipping_terms.append(shipping)
            else:
                shipping = 0.00
                shipping_terms.append(shipping)

            brand_name = con.find_all('a', class_="item-brand")[0].img["title"]
            product_brands.append(brand_name)

        except (IndexError, ValueError) as e:
            # if there's no item_brand container, take the Brand from product details
            product_brands.append(con.find_all('a', class_="item-title")[0].text.split()[0])
            #print(f"{e} block 1")

        try:
            current_promo = con.find_all("p", class_="item-promo")[0].text
            promotions.append(current_promo)
        except:
            promotions.append('null')
            #print(f"{e} block 2")
        try:
            price = con.find_all('li', class_="price-current")[0].text.split()[0].replace('$','').replace(',', '')
            prices.append(price)
        except (IndexError, ValueError) as e:
            prices.append('null')
            #print(f"{e} block 3")


    df = pd.DataFrame({
    'item_number': item_numbers,
    'brand': product_brands,
    'model_specifications': product_models,
    'price': prices,
    'current_promotions': promotions,
    'shipping': shipping_terms,
    'page_number': page_nums,
    'product_links': product_links,
    'image_link': images
    })
    df['general_category'] = current_page_soup.find_all('div', class_="nav-x-body-top-bar fix")[0].text.split('\n')[5]
    df['product_category'] = current_page_soup.find_all('h1', class_="page-title-text")[0].text
    # rearrange columns
    df = df[['item_number', 'general_category','product_category', 'page_number' ,'brand','model_specifications' ,'current_promotions' ,'price' ,'shipping' ,'product_links','image_link']]
    global product_category
    product_category = df['product_category'].unique()[0]
    # eliminate special characters in a string if it exists
    product_category = ''.join(e for e in product_category if e.isalnum())
    
    #return_list.append(product_category)
    global items_scraped
    items_scraped = len(df['model_specifications'])
    
    df.to_csv(f'./processing/{current_date}_{product_category}_{items_scraped}_scraped_page{turn_page}.csv')
    return items_scraped, product_category
    
#df.head()
#newegg_page_scraper(containers, turn_page)

In [ ]:
#print(containers[1].find_all('a', class_="item-brand")[0].img["title"])

In [ ]:
#######################################################################################


# create a function to return results pages, if exists, otherwise just scrape one page
def results_pages():
    # Use BeautifulSoup to extract the total results page number
    results_pages = current_page_soup.find_all('span', class_="list-tool-pagination-text")[0].text.strip()
    #print(results_pages)
    # Find and extract total pages + and add 1 to ensure proper length of total pages
    global total_results_pages
    total_results_pages = int(re.split("/", results_pages)[1]) #+ 2 # need to add 2 b/c 'range(inclusive, exclusive)'
    #========================================= need to remember to +2, and remove -30
    #print(total_results_pages)
    total_results_pages = total_results_pages ## ### ## ###
    
    return total_results_pages
#results_pages()

In [ ]:
# Working
def concatenate(total_results_pages):
    
    path = f'./processing\\'
    scraped_pages = glob.glob(path + "/*.csv")
    concatenate_pages = []
    counter = 0
    for page in scraped_pages:
        df = pd.read_csv(page, index_col=0, header=0)
        concatenate_pages.append(df)

    compiled_data = pd.concat(concatenate_pages, axis=0, ignore_index=True)
    total_items_scraped = len(compiled_data['brand']) # can replace this counter by creating class objects everytime it scrapes
    concatenated_output = compiled_data.to_csv(f"./finished_outputs/{current_date}_{total_items_scraped}_scraped_{total_results_pages}_pages_.csv")
    return concatenated_output


In [ ]:
# total_results_pages = 4
# concatenate(total_results_pages)

In [ ]:
# THis is working
def clean_processing_fldr():
    # delete all files in the 'processing folder'
    path = f'./processing\\'
    scraped_pages = glob.glob(path + "/*.csv")
    if len(scraped_pages) < 1:
        print("There are no files in the folder to clear. ")
    else:
        print(f"Clearing out a total of {len(scraped_pages)} scraped pages in the processing folder... ")
        clear_processing_files = []
        for page in scraped_pages:
            os.remove(page)
        
    print('Clearing of "Processing" folder complete. ')

In [ ]:
# webscrape first page, then run page turner, then scraper for every page thereafter
# 
    

In [ ]:
# learning lesson is you can't call a function within a function

def page_turner(total_results_pages):
    # This is "NEXT PAGE BUTTON CLICK" - This loops thru the total amount of pages by clicking the next page button
    for turn_page in range(1, total_results_pages):
        # set the current url as the target page (aiming the boomerang)
        target_url = browser.url

        # Use Request.get() - throw the boomerang at the target, retrieve the info, & return back to requestor
        response_target = requests.get(target_url)
        #response

        # Use BeautifulSoup to read grab all the HTML using the lxml parser
        target_page_soup = soup(response_target.text, 'html.parser')

        # Use BeautifulSoup to extract the total results page number
        #results_pages = current_page_soup.find_all('span', class_="list-tool-pagination-text")[0].text.strip()
        
        results_pages = target_page_soup.find_all('span', class_="list-tool-pagination-text")[0].text.strip()
        #=========================================================
        containers = target_page_soup.find_all("div", class_="item-container")
        
        newegg_page_scraper(containers, turn_page)
        
        #for i in range(total_results_pages):
        x = random.randint(3, 25)
        print(f"{turn_page}) | SLEEPING FOR SECONDS: {x} ")
        time.sleep(x)
            
        browser.find_by_xpath('//*[@id="bodyArea"]/section/div/div/div[2]/div/div/div/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[2]/button').click()

    browser.quit()
# concatenate(total_results_pages)
# clean_processing_fldr()
# # clear out processing folder function here - as delete everything to prevent clutter


# print(f'WebScraping Complete! All {total_results_pages} have been scraped and saved as {current_date}_{product_category}_scraped_{total_results_pages}_pages_.csv in the "finished_outputs" folder')
# print('Thank you and hope you found this useful!')

In [ ]:
# scrape_again = 'y'
# while scrape_again =='y':
#     print("=== NewEgg.Com WebScraper Beta ===")
#     print("=="*30)
#     print("Instructions:")
#     print("(1) Go to www.newegg.com, search for your laptop requirements (e.g. brand and specifications) ")
#     print("(2) Copy and paste the url from your exact search ")
#     print('(3) Activate or Disable the "Head View", webscraper bots point of view ')
#     print('(4) Check the "final_output folder when the webscraper bot is done scraping "')
#     print("")
# # NEED TO CREATE LAST INPUT TO LOOP THIS WHOLE THING AGAIN.
# #Maybe use pandas to provide summary of data
# #Export images of box and whisker plots using statistics by brand

return_dt()

executable_path = {'executable_path': './chromedriver.exe'}

url = input("Step 1) Please copy and paste your laptop query that you want to webscrape, and press enter: ")
head = ''
browser =''
head_on_off(executable_path)
response = requests.get(current_url)
#response

current_page_soup = soup(response.text, 'html.parser')
current_page_soup.find_all("div", class_="item-container")
scrappable_y_n(current_page_soup)

# Are there any pop ups / safe to proceed?
safe_proceed_y_n = input(f'The Break Pedal: Answer any robot queries by NewEgg. Enter "y" when you are ready to proceed. ')
if safe_proceed_y_n == 'y':
    print(f'Proceeding with webscrape... ')
else:
    print("Quitting browser. You will need to press ctrl + c to quit, and then restart the program to try again. ")
    browser.quit()
#newegg_page_scraper(containers)

# will need to UNCOMMENT AFTER
results_pages()

#page_turner(total_results_pages)

#total_results_pages = 5
for turn_page in range(1, total_results_pages):
    # set the current url as the target page (aiming the boomerang)
    target_url = browser.url

    # Use Request.get() - throw the boomerang at the target, retrieve the info, & return back to requestor
    response_target = requests.get(target_url)
    #response

    # Use BeautifulSoup to read grab all the HTML using the lxml parser
    target_page_soup = soup(response_target.text, 'html.parser')

    # Use BeautifulSoup to extract the total results page number
    #results_pages = current_page_soup.find_all('span', class_="list-tool-pagination-text")[0].text.strip()

    results_pages = target_page_soup.find_all('span', class_="list-tool-pagination-text")[0].text.strip()
    #=========================================================
    containers = target_page_soup.find_all("div", class_="item-container")

    newegg_page_scraper(containers, turn_page)

    #for i in range(total_results_pages):
    x = random.randint(3, 25)
    print(f"{turn_page}) | SLEEPING FOR {x} SECONDS ")
    time.sleep(x)

    browser.find_by_xpath('//*[@id="bodyArea"]/section/div/div/div[2]/div/div/div/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[2]/button').click()

browser.quit()


##################################################################
concat_y_n = input(f'All {total_results_pages} pages have been saved in the "processing" folder (1 page = csv files). Would you like for us concatenate all the files into one? Enter "y", if so. Otherwise, enter anykey to exit the program. ')
if concat_y_n == 'y':
    concatenate(total_results_pages)
    print(f'WebScraping Complete! All {total_results_pages} have been scraped and saved as {current_date}_{product_category}_scraped_{total_results_pages}_pages_.csv in the "finished_outputs" folder')

# clear out processing folder function here - as delete everything to prevent clutter
clear_processing_y_n = input(f'The "processing" folder has {total_results_pages} csv files of each page that was scraped. Would you like to clear the files? Enter "y", if so. Otherwise, enter anykey to exit the program. ')
if clear_processing_y_n == 'y':
    clean_processing_fldr()


print('Thank you and hope you found this useful!')